# Dependencies

In [1]:
import kaggle

# Download and load database

## Extract data from Kaggle ID dataset

For this case: https://www.kaggle.com/datasets/shivamb/netflix-shows

In [9]:
kaggle_dataset_id = "shivamb/netflix-shows"

In [10]:
kaggle.api.authenticate()
kaggle.api.dataset_download_files(kaggle_dataset_id, path='./', unzip=True)

## Save to duckdb

db_path = "data.duckdb"
table_name = "netflix_shows"
save_to_duckdb(df, table_name, db_path)

# Visualize database

In [2]:
%load_ext sql
%sql duckdb://

The 'toml' package isn't installed. To load settings from the pyproject.toml file, install with: pip install toml

Connecting to 'duckdb://'

In [3]:
%%sql
SELECT * FROM netflix_titles.csv LIMIT 5

Running query in 'duckdb://'

show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,None,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmmaker Kirsten Johnson stages his death in inventive and comical ways to help them both face the inevitable."
s2,TV Show,Blood & Water,None,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thabang Molaba, Dillon Windvogel, Natasha Thahane, Arno Greeff, Xolile Tshabalala, Getmore Sithole, Cindy Mahlangu, Ryle De Morny, Greteli Fincham, Sello Maake Ka-Ncube, Odwa Gwanya, Mekaila Mathys, Sandi Schultz, Duane Williams, Shamilla Miller, Patrick Mofokeng",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town teen sets out to prove whether a private-school swimming star is her sister who was abducted at birth."
s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabiha Akkari, Sofia Lesaffre, Salim Kechiouche, Noureddine Farihi, Geert Van Rampelberg, Bakary Diombera",None,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Action & Adventure","To protect his family from a powerful drug lord, skilled thief Mehdi and his expert team of robbers are pulled into a violent and deadly turf war."
s4,TV Show,Jailbirds New Orleans,None,None,None,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down among the incarcerated women at the Orleans Justice Center in New Orleans on this gritty reality series."
s5,TV Show,Kota Factory,None,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam Khan, Ahsaas Channa, Revathi Pillai, Urvi Singh, Arun Kumar",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV Comedies","In a city of coaching centers known to train India’s finest collegiate minds, an earnest but unexceptional student and his friends navigate campus life."


# Insights

### Count the number of Movies and TV Shows

In [4]:
query = f"""
SELECT 
    type, 
    COUNT(*) AS count
FROM 
    netflix_titles.csv
GROUP BY 
    type;
"""
# Use JupySQL magic %sql to execute the query
%sql {{query}}


Running query in 'duckdb://'

type,count
Movie,6131
TV Show,2676


### Find the top 5 countries with the most content

In [5]:
query = f"""
SELECT 
    country, 
    COUNT(*) AS count
FROM 
    netflix_titles.csv
GROUP BY 
    country
ORDER BY 
    count DESC
LIMIT 5;
"""
# Use JupySQL magic %sql to execute the query
%sql {{query}}

Running query in 'duckdb://'

country,count
United States,2818
India,972
None,831
United Kingdom,419
Japan,245


### Average duration of Movies

In [6]:
query = f"""
SELECT 
    AVG(CAST(SUBSTR(duration, 1, LENGTH(duration) - 4) AS INT)) AS avg_duration_minutes
FROM 
    netflix_titles.csv
WHERE 
    type = 'Movie';

"""
# Use JupySQL magic %sql to execute the query
%sql {{query}}

Running query in 'duckdb://'

avg_duration_minutes
99.57718668407311


### Count of content added in each year

In [7]:
query = f"""
SELECT 
    release_year, 
    COUNT(*) AS count
FROM 
    netflix_titles.csv
GROUP BY 
    release_year
ORDER BY 
    release_year;

"""
# Use JupySQL magic %sql to execute the query
%sql {{query}}

Running query in 'duckdb://'

release_year,count
1925,1
1942,2
1943,3
1944,3
1945,4
1946,2
1947,1
1954,2
1955,3
1956,2


In [8]:


query = f"""
SELECT 
    listed_in, 
    COUNT(*) AS count
FROM 
    netflix_titles.csv
GROUP BY 
    listed_in
ORDER BY 
    count DESC
LIMIT 5;

"""
# Use JupySQL magic %sql to execute the query
%sql {{query}}

Running query in 'duckdb://'

listed_in,count
"Dramas, International Movies",362
Documentaries,359
Stand-Up Comedy,334
"Comedies, Dramas, International Movies",274
"Dramas, Independent Movies, International Movies",252
